<a href="https://colab.research.google.com/github/jasonstonemed/Home_Sales/blob/main/Home_Sales_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install findspark

# Import packages
import os
import pyspark
from pyspark.sql import SparkSession
import time
import findspark
findspark.init()
from pyspark import *

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=ca371c2db21c4fd5165b20d6e7fa53b1c982af5a76a50e828784fd9a5463521e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/l

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

# Create dataframe
home_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True, inferSchema=True)
home_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_sales')



In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Run query
spark.sql("""
SELECT
    YEAR(date) as Year,
    ROUND(AVG(price), 2) as Average_Price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    Year
""").show()




+----+-------------+
|Year|Average_Price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Run query
spark.sql("""
SELECT
    date_built as Year_Built,
    ROUND(AVG(price), 2) as Average_Price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
""").show()



+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Run query
spark.sql("""
SELECT
    date_built as Year_Built,
    ROUND(AVG(price), 2) as Average_Price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
""").show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Capture start time
start_time = time.time()

# Run query
spark.sql("""
SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.2427020072937012 seconds ---


In [9]:
# 7. Cache the the temporary table home.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")


True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Start counter
cached_start_time = time.time()

# Run query from cache
query = """
SELECT
  view,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""

# Show query
spark.sql(query).show()

#Print time
print("--- %s seconds ---" % (time.time() - cached_start_time))

# Cached runtime
start_time_cached = time.time()
result_cached = spark.sql(query)
end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached

#Uncached runtime
start_time_uncached = time.time()
result_uncached = spark.sql(query)
end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached

print("Runtime with Cached Data:", runtime_cached, "seconds")
print("Runtime with Uncached Data:", runtime_uncached, "seconds")



+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9746148586273193 seconds ---
Runtime with Cached Data: 0.038355350494384766 seconds
Runtime with Uncached Data: 0.04568171501159668 seconds


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet('delayed_partitioned')

In [24]:
# 12. Create a temporary SQL table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales")


In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Start timer
parq_start_time = time.time()

# Run query from 'home_sales'
query = """
SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""

# Show query
spark.sql(query).show()

print("Parquet Time:--- %s seconds ---" % (time.time() - parq_start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

Parquet Time:--- 0.739875078201294 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False